In [1]:
import feedparser
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

clean_path = "../data/clean/"
raw_path = "../data/raw/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
pdf_path = "../PDF/"

pd.set_option('display.max_colwidth', 255)
pd.set_option('display.max_rows',None)
url = "https://feeds.feedburner.com/Setorth-form45-en"

today = date.today()
year = 2021
mmdd_str = today.strftime('%m%d')
mmdd_str

'0216'

In [2]:
today = date(2022, 2, 15)
mmdd_str = today.strftime('%m%d')
mmdd_str

'0215'

In [3]:
rss_source = feedparser.parse(url)
f45_number = len(rss_source.entries)
print("Number of F45: ", f45_number)

Number of F45:  18


In [4]:
f45_items = []

for x in range(f45_number):
    f45_content = rss_source.entries[x]
    f45_item = {}
    
    print("\n----------------------------------\n")
    
    print("F45: " + str(x))
    title_ary = f45_content.title.partition(' ')
    f45_item['name'] = title_ary[0].strip() 
    print("Title: ", f45_item['name'])  
    f45_item['year'] = year
    print("Year: ", f45_item['year'])      
    qtr_ary = title_ary[2].partition(' (F45)')
    f45_item['quarter'] = qtr_ary[0][-1]    
    print("Quarter: ", f45_item['quarter'])    
    f45_item['link'] = f45_content.link
    print("Link: ", f45_item['link'])
    f45_item['published'] = f45_content.published
    print("Published: ", f45_item['published'])  
    f45_items.append(f45_item)


----------------------------------

F45: 0
Title:  JTS
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814174730
Published:  Tue, 15 Feb 2022 22:29:10 +0700

----------------------------------

F45: 1
Title:  KWC
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814172640
Published:  Tue, 15 Feb 2022 20:47:20 +0700

----------------------------------

F45: 2
Title:  THG
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814171270
Published:  Tue, 15 Feb 2022 19:23:03 +0700

----------------------------------

F45: 3
Title:  AMANAH
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814169690
Published:  Tue, 15 Feb 2022 18:47:44 +0700

----------------------------------

F45: 4
Title:  OR
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?la

In [5]:
df = pd.DataFrame(f45_items)
df[['name','year','quarter','published']]

,name,year,quarter,published
0,JTS,2021,y,"Tue, 15 Feb 2022 22:29:10 +0700"
1,KWC,2021,y,"Tue, 15 Feb 2022 20:47:20 +0700"
2,THG,2021,y,"Tue, 15 Feb 2022 19:23:03 +0700"
3,AMANAH,2021,y,"Tue, 15 Feb 2022 18:47:44 +0700"
4,OR,2021,y,"Tue, 15 Feb 2022 18:17:55 +0700"
5,TOP,2021,y,"Tue, 15 Feb 2022 18:08:21 +0700"
6,DELTA,2021,y,"Tue, 15 Feb 2022 17:53:24 +0700"
7,SF,2021,y,"Tue, 15 Feb 2022 17:41:35 +0700"
8,MPIC,2021,y,"Tue, 15 Feb 2022 17:36:30 +0700"
9,GIFT,2021,y,"Tue, 15 Feb 2022 17:15:02 +0700"


In [6]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == 'y','quarter'] = 4
#df.replace({"quarter" : "1"}, "3", inplace=True)
df['quarter'].value_counts

<bound method IndexOpsMixin.value_counts of 0     4
1     4
2     4
3     4
4     4
5     4
6     4
7     4
8     4
9     4
10    4
11    4
12    e
13    4
14    e
15    4
16    4
17    4
Name: quarter, dtype: object>

In [7]:
df.loc[df.quarter == 'e','quarter'] = 4
df['quarter'].value_counts

<bound method IndexOpsMixin.value_counts of 0     4
1     4
2     4
3     4
4     4
5     4
6     4
7     4
8     4
9     4
10    4
11    4
12    4
13    4
14    4
15    4
16    4
17    4
Name: quarter, dtype: object>

In [ ]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == '1','year'] = 2022
#df.replace({"quarter" : "1"}, "3", inplace=True)

df

In [8]:
df.shape

(18, 5)

In [9]:
### First equals to latest published pdf file
df = df.drop_duplicates(subset='name', keep='first')
df.shape

(17, 5)

In [10]:
file_name = 'F45-RAW-' + mmdd_str + '.csv'
raw_file = raw_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[['name','year','quarter','published']].to_csv(output_file, header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(box_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(one_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(raw_file,    header=True, index=False, sep=',')

In [11]:
sql = '''
SELECT *
FROM exempts
ORDER BY name'''
df_exempts = pd.read_sql(sql, conlt)
df_exempts.shape[0]

299

In [12]:
df_merge = pd.merge(df, df_exempts, on='name', how='outer', indicator=True)
df_merge.shape

(313, 7)

### Tickers that won't be input

In [13]:
in_exempts = df_merge.loc[
    df_merge['_merge'] == 'both',
    ['name','year','quarter','published','link']
]
in_exempts.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
1,KWC,2021.0,4,"Tue, 15 Feb 2022 20:47:20 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814172640
9,GIFT,2021.0,4,"Tue, 15 Feb 2022 17:15:02 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814158250
13,PT,2021.0,4,"Tue, 15 Feb 2022 12:39:57 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814142630


In [14]:
in_exempts.sort_values(by=['published'],ascending=[False]).shape[0]

3

In [15]:
df_out = df_merge.loc[
    df_merge['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]
df_out.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
0,JTS,2021.0,4,"Tue, 15 Feb 2022 22:29:10 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814174730
2,THG,2021.0,4,"Tue, 15 Feb 2022 19:23:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814171270
3,AMANAH,2021.0,4,"Tue, 15 Feb 2022 18:47:44 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814169690
4,OR,2021.0,4,"Tue, 15 Feb 2022 18:17:55 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166960
5,TOP,2021.0,4,"Tue, 15 Feb 2022 18:08:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166040
6,DELTA,2021.0,4,"Tue, 15 Feb 2022 17:53:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814164470
7,SF,2021.0,4,"Tue, 15 Feb 2022 17:41:35 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814162280
8,MPIC,2021.0,4,"Tue, 15 Feb 2022 17:36:30 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814161110
10,GC,2021.0,4,"Tue, 15 Feb 2022 17:13:59 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814158090
11,DCC,2021.0,4,"Tue, 15 Feb 2022 13:09:31 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814144370


In [16]:
#df_out = df_out.drop(df_out.index[df_out['name'] == "SCC"])
df_out.shape[0]

14

In [17]:
sql = '''
SELECT *
FROM tickers
ORDER BY name'''
df_tickers = pd.read_sql(sql, conlt)
df_tickers.shape

(403, 9)

In [18]:
df_merge2 = pd.merge(df_out, df_tickers, on='name', how='outer', indicator=True)
df_merge2.shape

(406, 14)

### There are no ticker records

In [19]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
]

,name,year,quarter,published,link,id,market
7,MPIC,2021.0,4,"Tue, 15 Feb 2022 17:36:30 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814161110,NaN,NaN
11,KBSPIF,2021.0,4,"Tue, 15 Feb 2022 12:39:40 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814142610,NaN,NaN
13,STC,2021.0,4,"Tue, 15 Feb 2022 08:26:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814136410,NaN,NaN


### There are ticker records

In [20]:
df_out2 = df_merge2.loc[
    df_merge2['_merge'] == 'both',
    ['name','year','quarter','published','link','id','market']
]
df_out2

,name,year,quarter,published,link,id,market
0,JTS,2021.0,4,"Tue, 15 Feb 2022 22:29:10 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814174730,239.0,SET
1,THG,2021.0,4,"Tue, 15 Feb 2022 19:23:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814171270,661.0,SETCLMV / SETWB
2,AMANAH,2021.0,4,"Tue, 15 Feb 2022 18:47:44 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814169690,19.0,sSET
3,OR,2021.0,4,"Tue, 15 Feb 2022 18:17:55 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166960,715.0,SET50 / SETCLMV
4,TOP,2021.0,4,"Tue, 15 Feb 2022 18:08:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166040,555.0,SET50 / SETCLMV / SETTHSI
5,DELTA,2021.0,4,"Tue, 15 Feb 2022 17:53:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814164470,138.0,SET
6,SF,2021.0,4,"Tue, 15 Feb 2022 17:41:35 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814162280,438.0,SET
8,GC,2021.0,4,"Tue, 15 Feb 2022 17:13:59 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814158090,183.0,SET
9,DCC,2021.0,4,"Tue, 15 Feb 2022 13:09:31 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814144370,135.0,SET
10,EGATIF,2021.0,4,"Tue, 15 Feb 2022 13:04:30 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814144120,156.0,SET


In [21]:
df_out2 = df_out2[df_out2.year.notnull()]
df_out2

,name,year,quarter,published,link,id,market
0,JTS,2021.0,4,"Tue, 15 Feb 2022 22:29:10 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814174730,239.0,SET
1,THG,2021.0,4,"Tue, 15 Feb 2022 19:23:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814171270,661.0,SETCLMV / SETWB
2,AMANAH,2021.0,4,"Tue, 15 Feb 2022 18:47:44 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814169690,19.0,sSET
3,OR,2021.0,4,"Tue, 15 Feb 2022 18:17:55 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166960,715.0,SET50 / SETCLMV
4,TOP,2021.0,4,"Tue, 15 Feb 2022 18:08:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166040,555.0,SET50 / SETCLMV / SETTHSI
5,DELTA,2021.0,4,"Tue, 15 Feb 2022 17:53:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814164470,138.0,SET
6,SF,2021.0,4,"Tue, 15 Feb 2022 17:41:35 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814162280,438.0,SET
8,GC,2021.0,4,"Tue, 15 Feb 2022 17:13:59 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814158090,183.0,SET
9,DCC,2021.0,4,"Tue, 15 Feb 2022 13:09:31 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814144370,135.0,SET
10,EGATIF,2021.0,4,"Tue, 15 Feb 2022 13:04:30 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814144120,156.0,SET


In [22]:
df_out2['year'] = df_out2['year'].astype(int)
df_out2['quarter'] = df_out2['quarter'].astype(int)
df_out2

,name,year,quarter,published,link,id,market
0,JTS,2021,4,"Tue, 15 Feb 2022 22:29:10 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814174730,239.0,SET
1,THG,2021,4,"Tue, 15 Feb 2022 19:23:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814171270,661.0,SETCLMV / SETWB
2,AMANAH,2021,4,"Tue, 15 Feb 2022 18:47:44 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814169690,19.0,sSET
3,OR,2021,4,"Tue, 15 Feb 2022 18:17:55 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166960,715.0,SET50 / SETCLMV
4,TOP,2021,4,"Tue, 15 Feb 2022 18:08:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166040,555.0,SET50 / SETCLMV / SETTHSI
5,DELTA,2021,4,"Tue, 15 Feb 2022 17:53:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814164470,138.0,SET
6,SF,2021,4,"Tue, 15 Feb 2022 17:41:35 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814162280,438.0,SET
8,GC,2021,4,"Tue, 15 Feb 2022 17:13:59 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814158090,183.0,SET
9,DCC,2021,4,"Tue, 15 Feb 2022 13:09:31 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814144370,135.0,SET
10,EGATIF,2021,4,"Tue, 15 Feb 2022 13:04:30 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814144120,156.0,SET


In [23]:
file_name = 'F45-CLEAN-' + mmdd_str + '.csv'
clean_file = clean_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(output_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(clean_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(box_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(one_file, header=True, index=False, sep=',')

In [24]:
sql = '''
SELECT * 
FROM epss
WHERE year = 2021'''
df_epss = pd.read_sql(sql, conlt)
df_epss.shape

(1053, 14)

In [25]:
df_merge3 = pd.merge(df_out2, df_epss, on=['name','year','quarter'], how='outer', indicator=True)
df_merge3.shape

(1064, 19)

### Already input, display profit amt & eps to check with new F45

In [26]:
df_merge3[df_merge3['_merge'] == 'both']

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge


In [27]:
df_inp2 = df_merge3[df_merge3['_merge'] == 'left_only']
df_inp3 = df_inp2.copy()
df_inp3

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge
0,JTS,2021,4,"Tue, 15 Feb 2022 22:29:10 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814174730,239.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,THG,2021,4,"Tue, 15 Feb 2022 19:23:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814171270,661.0,SETCLMV / SETWB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,AMANAH,2021,4,"Tue, 15 Feb 2022 18:47:44 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814169690,19.0,sSET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,OR,2021,4,"Tue, 15 Feb 2022 18:17:55 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166960,715.0,SET50 / SETCLMV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,TOP,2021,4,"Tue, 15 Feb 2022 18:08:21 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166040,555.0,SET50 / SETCLMV / SETTHSI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,DELTA,2021,4,"Tue, 15 Feb 2022 17:53:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814164470,138.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6,SF,2021,4,"Tue, 15 Feb 2022 17:41:35 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814162280,438.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7,GC,2021,4,"Tue, 15 Feb 2022 17:13:59 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814158090,183.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
8,DCC,2021,4,"Tue, 15 Feb 2022 13:09:31 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814144370,135.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9,EGATIF,2021,4,"Tue, 15 Feb 2022 13:04:30 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814144120,156.0,SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [28]:
df_inp3['year'] = df_inp3['year'].astype(str)
df_inp3['quarter'] = df_inp3['quarter'].astype(str)
inp3_str = df_inp3.name+' '+df_inp3.year+' '+df_inp3.quarter+' '+df_inp3.link
inp3_str

0        JTS 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814174730
1        THG 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814171270
2     AMANAH 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814169690
3         OR 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166960
4        TOP 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814166040
5      DELTA 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814164470
6         SF 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814162280
7         GC 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814158090
8        DCC 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16448814144370
9     EGATIF 2021 4 https://www.set.or.th/set/